# Connecting

In [1]:
import os

# setting working dir
os.chdir("../src")

In [2]:
from elasticsearch_db.elasticsearch import elastic_conection
from elasticsearch_db.elasticsearch import get_nlp_model

es = elastic_conection()

In [3]:
workspace_id = "dc1e7b3d-9137-4a20-a99c-d0d2029ef170"
workspace, exist = get_nlp_model(es, workspace_id=workspace_id)
exist

True

# Data Sourcing

In [4]:
from data_pipeline.data_sourcing import get_data

In [5]:
data = get_data(workspace=workspace, es=es)

In [6]:
print(len(data))
data[0]

206


{'intent': 'Inativar_Posição',
 'examples': [{'text': 'Como realizo a inativação de uma posição de minha estrutura?'},
  {'text': 'Como realizo a reativação de uma posição em minha estrutura?'},
  {'text': 'Em quanto tempo a inativação de uma posição é efetivada?'},
  {'text': 'Gostaria de fazer a inativação de uma posição, como faço?'},
  {'text': 'Realizei a inativação de uma posição e ela continua visível?'}],
 'description': 'tema Dúvidas sobre reembolso, adiantamentos e prestação de contas',
 'doc_type': 'intent'}